# Neural Network Modelling for Glacier Mass Balance Prediction

This notebook implements **neural network models** for predicting glacier point mass balance measurements using meteorological and topographical features. The notebook supports two main modelling approaches:

##  **Modelling Strategies**

### 1. **Regional Modelling (French Alps Only)**
- Train and test exclusively on French Alps glacier data
- Uses glacier-based train/test splits to ensure spatial generalization

### 2. **Cross-Regional Modelling (Switzerland → French Alps)**
- Combines data from Swiss glaciers and French Alps glaciers
- Can be used as either a cross-regional model, training exclusively on Swiss data and testing on French Alps data
- or as a baseline for transfer learning models, training on Swiss data and a subset of Icelandic data and testing on remaining Icelandic data

---

##  **Prerequisites**
- French Alps glacier dataset from `../1.1. France-prepro.ipynb`
- Swiss glacier dataset from `regions/Switzerland/1.1. GLAMOS-prepro.ipynb`
- ERA5 climate data for both regions from `../1.2. ERA5Land-prepro.ipynb`
---

### Feature Definitions

**Climate Features (ERA5 Reanalysis):**
- `t2m`: 2-meter temperature
- `tp`: Total precipitation
- `slhf`/`sshf`: Surface heat fluxes
- `ssrd`: Surface solar radiation downwards
- `fal`: Albedo
- `str`: Surface net thermal radiation
- `u10`/`v10`: Wind components

**Topographical Features (OGGM):**
- `aspect`/`slope`: Terrain geometry
- `hugonnet_dhdt`: Ice thickness changes
- `consensus_ice_thickness`: Ice depth
- `millan_v`: Ice surface velocity
- `elevation_difference`: Measurement elevation − ERA5-Land grid cell elevation

In [ ]:
import sys, os
sys.path.append(os.path.join(os.getcwd(), '../../')) # Add root of repo to import MBM

import pandas as pd
import numpy as np
import warnings
import re
import matplotlib.pyplot as plt
import seaborn as sns
from cmcrameri import cm
import massbalancemachine as mbm
import logging
import torch.nn as nn
from skorch.helper import SliceDataset
from datetime import datetime
from skorch.callbacks import EarlyStopping, LRScheduler, Checkpoint
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset
import pickle 
from scipy import stats
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from regions.French_Alps.scripts.helpers import *
from regions.French_Alps.scripts.glacioclim_preprocess import *
from regions.French_Alps.scripts.plots import *
from regions.French_Alps.scripts.config_FR import *
from regions.French_Alps.scripts.nn_helpers import *
from regions.French_Alps.scripts.NN_networks import *

from regions.Switzerland.scripts.dataset import process_or_load_data, get_CV_splits
from regions.Switzerland.scripts.models import evaluate_NN_and_group_predictions,compute_seasonal_scores
from regions.Switzerland.scripts.plotting import plot_predictions_summary, plot_individual_glacier_pred

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

cfg = mbm.FranceConfig()
mbm.utils.seed_all(cfg.seed)
mbm.utils.free_up_cuda()
mbm.plots.use_mbm_style()

## 1. Regional Modelling (French Alps Only)

This approach trains neural networks exclusively on French Alps glacier data.

## Data Loading & Initial Processing

### Create French Alps Glacier Dataset

Start with point mass balance measurements and transform them to monthly format with ERA5 climate data.

In [ ]:
data_glacioclim = pd.read_csv(cfg.dataPath + path_PMB_GLACIOCLIM_csv +
                              'FR_wgms_dataset_all.csv')

print('Number of glaciers:', len(data_glacioclim['GLACIER'].unique()))
print('-------------------')
print('Number of glaciers:', len(data_glacioclim['GLACIER'].unique()))
print('Number of winter and annual samples:', len(data_glacioclim))
print('Number of annual samples:',
      len(data_glacioclim[data_glacioclim.PERIOD == 'annual']))
print('Number of winter samples:',
      len(data_glacioclim[data_glacioclim.PERIOD == 'winter']))


### Dataset Transformation to Monthly Format

Transform point mass balance data to monthly resolution and integrate with ERA5 climate variables.

In [ ]:
# Transform data to monthly format (run or load data):
paths = {
    'csv_path':
    cfg.dataPath + path_PMB_GLACIOCLIM_csv,
    'era5_climate_data':
    cfg.dataPath + path_ERA5_raw + 'era5_monthly_averaged_data_Alps.nc',
    'geopotential_data':
    cfg.dataPath + path_ERA5_raw + 'era5_geopotential_pressure_Alps.nc'
}

RUN = True
data_monthly = process_or_load_data(run_flag=RUN,
                                    df=data_glacioclim,
                                    paths=paths,
                                    cfg=cfg,
                                    vois_climate=VOIS_CLIMATE,
                                    vois_topographical=VOIS_TOPOGRAPHICAL,
                                    region_name="FR",
                                    region_id=11,
                                    output_file='FR_wgms_dataset_monthly.csv',
                                    add_pcsr=False)
# Create DataLoader
dataloader_gl = mbm.dataloader.DataLoader(cfg,
                                          data=data_monthly,
                                          random_seed=cfg.seed,
                                          meta_data_columns=cfg.metaData)


### Train/Test Splitting Strategy

**Spatial Generalization Approach:** Select test set based on glaciers, remaining glaciers will be the train set

In [ ]:
test_glaciers = ['FR4N01146D09+E06 Gebroulaz']

#test_glaciers = ['FR4N01083B21 Blanc']

# Regional 50%
#test_glaciers = ['FR4N01236A07 de Talefre','FR4N01235A08 dArgentiere', 'FR4N01146D09+E06 Gebroulaz']

# 5-10% or 653 IDs in training set
#test_glaciers = ['FR4N01235A08 dArgentiere' ,'FR4N01146D09+E06 Gebroulaz' ,'FR4N01236A01 Mer de Glace/Geant','FR4N01162B09+154D03 de Saint Sorlin', 'FR4N01083B21 Blanc']

# Ensure all test glaciers exist in the dataset
existing_glaciers = set(dataloader_gl.data.GLACIER.unique())
missing_glaciers = [g for g in test_glaciers if g not in existing_glaciers]

if missing_glaciers:
    print(
        f"Warning: The following test glaciers are not in the dataset: {missing_glaciers}"
    )

# Define training glaciers
train_glaciers = [i for i in existing_glaciers if i not in test_glaciers]
data_test = dataloader_gl.data[dataloader_gl.data.GLACIER.isin(test_glaciers)]

# Statistics prints
print('Size of test data:', len(data_test))
data_train = dataloader_gl.data[dataloader_gl.data.GLACIER.isin(
    train_glaciers)]
print('Size of train data:', len(data_train))
if len(data_train) == 0:
    print("Warning: No training data available!")
else:
    test_perc = (len(data_test) / len(data_train)) * 100
    print('Percentage of test size: {:.2f}%'.format(test_perc))

## CV Splits
splits, test_set, train_set = get_CV_splits(dataloader_gl,
                                            test_split_on='GLACIER',
                                            test_splits=test_glaciers,
                                            random_state=cfg.seed)

print('Train glaciers: ({}) {}'.format(len(train_set['splits_vals']),
                                       train_set['splits_vals']))
print('Test glaciers: ({}) {}'.format(len(test_set['splits_vals']),
                                      test_set['splits_vals']))

**Temporal Generalization Approach:** Withhold X of the last years for the testing set.

In [ ]:
test_years_count = 10

all_years = sorted(dataloader_gl.data['YEAR'].unique())

# Use the most recent years as test data
train_years = all_years[:-test_years_count]
test_years = all_years[-test_years_count:]

# Statistics prints
print('Size of test data:', len(test_years))
data_train = dataloader_gl.data[dataloader_gl.data.YEAR.isin(train_years)]
print('Size of train data:', len(train_years))
if len(train_years) == 0:
    print("Warning: No training data available!")
else:
    test_perc = (len(test_years) / len(train_years)) * 100
    print('Percentage of test size: {:.2f}%'.format(test_perc))

## CV Splits
splits, test_set, train_set = get_CV_splits(dataloader_gl,
                                            test_split_on='YEAR',
                                            test_splits=test_years,
                                            random_state=cfg.seed)

print('Train year: ({}) {}'.format(len(train_set['splits_vals']),
                                   train_set['splits_vals']))
print('Test years: ({}) {}'.format(len(test_set['splits_vals']),
                                   test_set['splits_vals']))

region_name = 'Prediction most recent 10 years'

### Training/Validation Split Strategy (80/20)

**Standard Approach:** Random 80/20 split across all available training data.

In [ ]:
# Validation and train split:
data_train = train_set['df_X']
data_train['y'] = train_set['y']
dataloader = mbm.dataloader.DataLoader(cfg, data=data_train)

train_itr, val_itr = dataloader.set_train_test_split(test_size=0.2)

# Get all indices of the training and valing dataset at once from the iterators. Once called, the iterators are empty.
train_indices, val_indices = list(train_itr), list(val_itr)

df_X_train = data_train.iloc[train_indices]
y_train = df_X_train['POINT_BALANCE'].values

# Get val set
df_X_val = data_train.iloc[val_indices]
y_val = df_X_val['POINT_BALANCE'].values

## B. Cross-Regional Transfer Learning (Switzerland → French Alps)

This approach uses the Swiss dataset to try and model French Alps glaciers.

### Create Combined Swiss and French Alps Glacier Dataset

Start with point mass balance measurements and transform them to monthly format with ERA5 climate data.

In [ ]:
# Read in
data_FR = pd.read_csv(cfg.dataPath + path_PMB_GLACIOCLIM_csv +
                      'FR_wgms_dataset_all.csv')
data_CH = pd.read_csv(cfg.dataPath + path_PMB_GLAMOS_csv +
                      'CH_wgms_dataset_all.csv')
# Adjust dfs to match
data_CH['GLACIER_ZONE'] = 'Placeholder'
data_CH['DATA_MODIFICATION'] = ''
data_CH = data_CH.drop(columns=['aspect_sgi', 'slope_sgi', 'topo_sgi'],
                       errors='ignore')

data_FR = data_FR[data_FR['PERIOD'] != 'summer']

# Merge FR with CH
data_FR_CH = pd.concat([data_FR, data_CH], axis=0).reset_index(drop=True)
display(len(data_FR_CH['GLACIER'].unique()))

In [ ]:
# Transform data to monthly format (run or load data):
paths = {
    'csv_path':
    cfg.dataPath + path_PMB_GLACIOCLIM_csv,
    'era5_climate_data':
    cfg.dataPath + path_ERA5_raw + 'era5_monthly_averaged_data_Alps.nc',
    'geopotential_data':
    cfg.dataPath + path_ERA5_raw + 'era5_geopotential_pressure_Alps.nc'
}

RUN = True
data_monthly_FR_CH = process_or_load_data(
    run_flag=RUN,
    df=data_FR_CH,
    paths=paths,
    cfg=cfg,
    vois_climate=VOIS_CLIMATE,
    vois_topographical=VOIS_TOPOGRAPHICAL,
    region_name="FR",
    region_id=11,
    add_pcsr=False,
    output_file='FR_CH_wgms_dataset_monthly_full.csv')

dataloader_gl = mbm.dataloader.DataLoader(cfg,
                                          data=data_monthly_FR_CH,
                                          random_seed=cfg.seed,
                                          meta_data_columns=cfg.metaData)
display(data_monthly_FR_CH.head(2))

### Train/Test Splitting Strategy

**Spatial Generalization Approach:** Select test set based on glaciers, remaining glaciers will be the train set

In [ ]:
#test_glaciers = list(data_FR['GLACIER'].unique())

# 5-10% or 653 IDs in train set
test_glaciers = [
    'FR4N01235A08 dArgentiere', 'FR4N01146D09+E06 Gebroulaz',
    'FR4N01236A01 Mer de Glace/Geant', 'FR4N01162B09+154D03 de Saint Sorlin',
    'FR4N01083B21 Blanc'
]

# Ensure all test glaciers exist in the dataset
existing_glaciers = set(dataloader_gl.data.GLACIER.unique())
missing_glaciers = [g for g in test_glaciers if g not in existing_glaciers]

if missing_glaciers:
    print(
        f"Warning: The following test glaciers are not in the dataset: {missing_glaciers}"
    )

# Define training glaciers
train_glaciers = list(data_CH['GLACIER'].unique())
data_test = dataloader_gl.data[dataloader_gl.data.GLACIER.isin(test_glaciers)]

# Statistics prints
print('Size of test data:', len(data_test))
data_train = dataloader_gl.data[dataloader_gl.data.GLACIER.isin(
    train_glaciers)]
print('Size of train data:', len(data_train))
if len(data_train) == 0:
    print("Warning: No training data available!")
else:
    test_perc = (len(data_test) / len(data_train)) * 100
    print('Percentage of test size: {:.2f}%'.format(test_perc))

## CV Splits
splits, test_set, train_set = get_CV_splits(dataloader_gl,
                                            test_split_on='GLACIER',
                                            test_splits=test_glaciers,
                                            random_state=cfg.seed)

print('Train glaciers: ({}) {}'.format(len(train_set['splits_vals']),
                                       train_set['splits_vals']))
print('Test glaciers: ({}) {}'.format(len(test_set['splits_vals']),
                                      test_set['splits_vals']))

### Training/Validation Split Strategy (80/20)

**Standard Approach:** Random 80/20 split across all available training data (Swiss + French Alps training glaciers).

In [ ]:
# Validation and train split:
data_train = train_set['df_X']
data_train['y'] = train_set['y']
dataloader = mbm.dataloader.DataLoader(cfg, data=data_train)

train_itr, val_itr = dataloader.set_train_test_split(test_size=0.2)

# Get all indices of the training and valing dataset at once from the iterators. Once called, the iterators are empty.
train_indices, val_indices = list(train_itr), list(val_itr)

df_X_train = data_train.iloc[train_indices]
y_train = df_X_train['POINT_BALANCE'].values

# Get val set
df_X_val = data_train.iloc[val_indices]
y_val = df_X_val['POINT_BALANCE'].values

### Training/Validation Split - Target Domain Focus (80/20)

**Domain-Aware Approach:** This split will only build the validation set from French data

- **Training Set:** All Swiss data + 80% of French Alps training glaciers
- **Validation Set:** 20% of French Alps training glaciers only

This ensures that the same validation set is used as in fine-tuning and layer freezing in the `../3.2.4 Train-ML-model-NN_progressive_transfer` Notebook.

In [ ]:
# Full training pool = CH + France subset
data_train = train_set['df_X'].copy()
data_train['y'] = train_set['y']

# France train_glaciers
france_train_glacier = [
    g for g in data_FR['GLACIER'].unique() if g not in test_glaciers
]
display('train glaciers from target domain: ', france_train_glacier)

# Find France subset within this pool
france_mask = data_train['GLACIER'].isin(france_train_glacier)
data_france = data_train.loc[france_mask]

# Split only the France subset
dataloader = mbm.dataloader.DataLoader(cfg, data=data_france)
train_itr, val_itr = dataloader.set_train_test_split(test_size=0.2)

france_train_idx = list(train_itr)
france_val_idx = list(val_itr)

# Training set = CH + France train portion
df_X_train = pd.concat([
    data_train.loc[~france_mask],  # all CH glaciers
    data_france.iloc[france_train_idx]  # France train glaciers
])
y_train = df_X_train['POINT_BALANCE'].values

# Validation set = France val portion only
df_X_val = data_france.iloc[france_val_idx]
y_val = df_X_val['POINT_BALANCE'].values

print("Train data glacier distribution:",
      df_X_train['GLACIER'].value_counts().head())
print("Val data glacier distribution:",
      df_X_val['GLACIER'].value_counts().head())
print("Train data shape:", df_X_train.shape)
print("Val data shape:", df_X_val.shape)

## Neural Network Configuration & Training

Implementing a **Multilayer perceptron deep neural network** designed specifically for glacier mass balance prediction

The following cell is optional and initializes a period_indicator variable, which allows the NN to keep track if the current point is from a winter or annual measurement.

Uncomment `#+ ['PERIOD_INDICATOR']` in the "Define features" cell if you want to use this feature.

In [ ]:
## Optional, don't run if you dont want to use period_indicator


def create_period_indicator(df):
    """Create numerical PERIOD_INDICATOR feature"""
    df = df.copy()
    df['PERIOD_INDICATOR'] = df['PERIOD'].map({'annual': 0, 'winter': 1})
    return df


# Apply to all datasets
df_X_train = create_period_indicator(df_X_train)
df_X_val = create_period_indicator(df_X_val)
test_set['df_X'] = create_period_indicator(test_set['df_X'])

print("PERIOD_INDICATOR created:")
print("Annual (0):", (df_X_train['PERIOD_INDICATOR'] == 0).sum())
print("Winter (1):", (df_X_train['PERIOD_INDICATOR'] == 1).sum())
print("Original PERIOD column preserved:", df_X_train['PERIOD'].unique())

Define features

In [ ]:
features_topo = [
    'ELEVATION_DIFFERENCE',
] + list(VOIS_TOPOGRAPHICAL)

feature_columns = features_topo + list(VOIS_CLIMATE)

cfg.setFeatures(feature_columns)

all_columns = feature_columns + cfg.fieldsNotFeatures

# Because CH has some extra columns, we need to cut those
df_X_train_subset = df_X_train[all_columns]
df_X_val_subset = df_X_val[all_columns]
df_X_test_subset = test_set['df_X'][all_columns]

print('Shape of training dataset:', df_X_train_subset.shape)
print('Shape of validation dataset:', df_X_val_subset.shape)
print('Shape of testing dataset:', df_X_test_subset.shape)
print('Running with features:', feature_columns)

assert all(train_set['df_X'].POINT_BALANCE == train_set['y'])

### Network Initialization & Hyperparameters

**NN Configuration:**
- **Learning Rate:** 0.001 with ReduceLROnPlateau scheduling
- **Batch Size:** 128 samples per gradient update  
- **Optimization:** Adam optimizer with L2 weight decay (1e-5)
- **Architecture:** [128, 128, 64, 32] hidden layers with 20% dropout
- **Regularization:** Batch normalization + early stopping (patience=15)

**Callbacks for Robust Training:**
- **Early Stopping:** Prevents overfitting by monitoring validation loss
- **Learning Rate Scheduler:** Reduces LR when validation loss plateaus

In [ ]:
early_stop = EarlyStopping(
    monitor='valid_loss',
    patience=15,
    threshold=1e-4,  # Optional: stop only when improvement is very small
)

lr_scheduler_cb = LRScheduler(policy=ReduceLROnPlateau,
                              monitor='valid_loss',
                              mode='min',
                              factor=0.5,
                              patience=5,
                              threshold=0.01,
                              threshold_mode='rel',
                              verbose=True)

dataset = dataset_val = None  # Initialized hereafter


def my_train_split(ds, y=None, **fit_params):
    return dataset, dataset_val


# param_init = {'device': 'cuda:0'}
param_init = {'device': 'cpu'}  # Use CPU for training
nInp = len(feature_columns)
print('Number of input features:', nInp)

params = {
    'lr': 0.0005,
    'batch_size': 128,
    'optimizer': torch.optim.Adam,
    'optimizer__weight_decay': 0.0,
    'module__hidden_layers': [128, 128, 64, 64, 32],
    'module__dropout': 0.2,
    'module__use_batchnorm': False,
}

args = {
    'module': FlexibleNetwork,
    'nbFeatures': nInp,
    'module__input_dim': nInp,
    'module__dropout': params['module__dropout'],
    'module__hidden_layers': params['module__hidden_layers'],
    'train_split': my_train_split,
    'batch_size': params['batch_size'],
    'verbose': 1,
    'iterator_train__shuffle': True,
    'lr': params['lr'],
    'max_epochs': 200,
    'optimizer': params['optimizer'],
    'optimizer__weight_decay': params['optimizer__weight_decay'],
    'module__use_batchnorm': params['module__use_batchnorm'],
    'callbacks': [
        ('early_stop', early_stop),
        ('lr_scheduler', lr_scheduler_cb),
    ]
}

custom_nn = mbm.models.CustomNeuralNetRegressor(cfg, **args, **param_init)

### Dataset Preparation for Neural Network Training

In [ ]:
months_head_pad, months_tail_pad = mbm.data_processing.utils.build_head_tail_pads_from_monthly_df(
    data_monthly)
_, month_pos = mbm.data_processing.utils._rebuild_month_index(
    months_head_pad, months_tail_pad)

print(months_head_pad, months_tail_pad)

features, metadata = mbm.data_processing.utils.create_features_metadata(
    cfg, df_X_train_subset)

features_val, metadata_val = mbm.data_processing.utils.create_features_metadata(
    cfg, df_X_val_subset)

# Define the dataset for the NN
dataset = mbm.data_processing.AggregatedDataset(
    cfg,
    features=features,
    metadata=metadata,
    months_head_pad=months_head_pad,
    months_tail_pad=months_tail_pad,
    targets=y_train)
dataset = mbm.data_processing.SliceDatasetBinding(SliceDataset(dataset, idx=0),
                                                  SliceDataset(dataset, idx=1))
print("train:", dataset.X.shape, dataset.y.shape)

dataset_val = mbm.data_processing.AggregatedDataset(
    cfg,
    features=features_val,
    metadata=metadata_val,
    months_head_pad=months_head_pad,
    months_tail_pad=months_tail_pad,
    targets=y_val)
dataset_val = mbm.data_processing.SliceDatasetBinding(
    SliceDataset(dataset_val, idx=0), SliceDataset(dataset_val, idx=1))
print("validation:", dataset_val.X.shape, dataset_val.y.shape)

### Model Training & Checkpointing

Set `TRAIN = True` to train new model.

In [ ]:
TRAIN = True
if TRAIN:
    custom_nn.seed_all()

    print("Training the model...")
    print('Model parameters:')
    for key, value in args.items():
        print(f"{key}: {value}")
    custom_nn.fit(dataset.X, dataset.y)
    # The dataset provided in fit is not used as the datasets are overwritten in the provided train_split function

    # Generate filename with current date
    current_date = datetime.now().strftime("%Y-%m-%d")
    model_filename = f"nn_model_{current_date}"

    # After Training: Best weights are already loaded
    # Save the model
    custom_nn.save_model(model_filename)

    # save params dic
    params_filename = f"nn_params_{current_date}.pkl"

    with open(f"models/{params_filename}", "wb") as f:
        pickle.dump(args, f)

Load previously trained model.

In [ ]:
# Load model and set to CPU
model_filename = "nn_model_2026-02-03.pt"  # Replace with actual date if needed

# read pickle with params
params_filename = "nn_params_2026-02-03.pkl"  # Replace with actual date if needed
with open(f"models/{params_filename}", "rb") as f:
    custom_params = pickle.load(f)

params = custom_params

args = {
    'module': FlexibleNetwork,
    'nbFeatures': nInp,
    'module__input_dim': nInp,
    'module__dropout': params['module__dropout'],
    'module__hidden_layers': params['module__hidden_layers'],
    'train_split': my_train_split,
    'batch_size': params['batch_size'],
    'verbose': 1,
    'iterator_train__shuffle': True,
    'lr': params['lr'],
    'max_epochs': 300,
    'optimizer': params['optimizer'],
    'optimizer__weight_decay': params['optimizer__weight_decay'],
    'module__use_batchnorm': params['module__use_batchnorm'],
    'callbacks': [
        ('early_stop', early_stop),
        ('lr_scheduler', lr_scheduler_cb),
    ]
}

loaded_model = mbm.models.CustomNeuralNetRegressor.load_model(
    cfg,
    model_filename,
    **{
        **args,
        **param_init
    },
)
loaded_model = loaded_model.set_params(device='cpu')
loaded_model = loaded_model.to('cpu')

grouped_ids, scores_NN, ids_NN, y_pred_NN = evaluate_NN_and_group_predictions(
    loaded_model, df_X_test_subset, test_set['y'], months_head_pad,
    months_tail_pad)
baseline_score = scores_NN['rmse']
print('Baseline RMSE:', baseline_score)

In [ ]:
scores_annual, scores_winter = compute_seasonal_scores(grouped_ids,
                                                       target_col='target',
                                                       pred_col='pred')
print("Annual scores:", scores_annual)
print("Winter scores:", scores_winter)

fig = plot_predictions_summary(
    grouped_ids=grouped_ids,
    scores_annual=scores_annual,
    scores_winter=scores_winter,
    ax_xlim=(-8, 6),
    ax_ylim=(-8, 6),
    color_annual=mbm.plots.COLOR_ANNUAL,
    color_winter=mbm.plots.COLOR_WINTER,
)